In [16]:
!pip install pandas folium geopandas geopy

In [26]:
import pandas as pd
import matplotlib as mpl
#import seaborn as sea
#import folium as fol
import sklearn as sci
import requests
import folium
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [18]:
!pip install sodapy

In [19]:
api_url = "https://data.fortworthtexas.gov/resource/k6ic-7kp7.csv" # Replace with actual API endpoint

response = requests.get(api_url) # For GET request
# response = requests.post(api_url, data=payload) # For POST request with data

csv_filename = "fort_worth_crime_data.csv"

with open(csv_filename, "wb") as file:
    file.write(response.content)

# Read CSV into a DataFrame
df = pd.read_csv(csv_filename)

# Display the first few rows
df.head()


,case_no_offense,case_no,reported_date,nature_of_call,from_date,offense,offense_desc,block_address,city,state,beat,division,councildistrict,attempt_complete,location_type,locationtypedescription,location_1
0,180094593-35A,180094593,2018-10-22T18:15:00.000,NARCOTIC VIOLATIONS,2018-10-22T18:15:00.000,35A,HSC 481.115(B) Poss CS PG 1 <1G 35A DRUG/NARCOTIC,700 CALIFORNIA PKWY N WB,FORT WORTH,TX,J13,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN
1,200085517-90Z,200085517,2020-11-03T13:24:00.000,HARASS,2020-08-12T05:18:00.000,90Z,PC 42.07(C) Harassment,1300 MARLBOROUGH DR,FORT WORTH,TX,I16,South,8.0,C,20.0,20 RESIDENCE/HOME,"\n, \n(32.640426597727426, -97.33851074771222)"
2,210012743-240,210012743,2021-02-20T20:27:59.000,AUTO THEFT,2021-02-20T20:27:59.000,240,PC 31.03(E)(4)(A)AUTO Theft>$1500<$20K (Auto t...,SANDY LN & EDERVILLE RD,FORT WORTH,TX,H15,East,5.0,C,18.0,18 PARKING LOT/GARAGE,NaN
3,190047946-35A,190047946,2019-06-04T01:33:00.000,NARCOTIC VIOLATIONS,2019-06-04T01:33:49.000,35A,HSC 481.117(B) Poss CS PG 3 <28G 35A DRUG/NARC...,OAK GROVE RD / E IH 20 FWY EB,NaN,TX,I17,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN
4,190031427-90D,190031427,2019-04-13T02:44:00.000,DWI,2019-04-13T02:44:37.000,90D,PC 49.04 Driving While Intoxicated 90D DRIVING...,WEST FWY SR WB / GREEN OAKS RD,NaN,TX,L11,NaN,NaN,C,13.0,13 HIGHWAY/ROAD/ALLEY,NaN


In [20]:
division_counts = df["division"].value_counts()
print(division_counts)

division
East         144
South        110
West         104
Northwest     69
Central       67
North         49
Name: count, dtype: int64


In [ ]:
geolocator = Nominatim(user_agent="fort_worth_geocoder")

# Function to get latitude and longitude from an address
def get_lat_long(address):
    try:
        location = geolocator.geocode(address + ", Fort Worth, TX")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

# Apply geocoding to a sample (to avoid rate limits, use a subset)
tqdm.pandas()
df[["latitude", "longitude"]] = df["block_address"].progress_apply(lambda x: pd.Series(get_lat_long(str(x))))

# Drop rows with missing coordinates
df = df.dropna(subset=["latitude", "longitude"])

 63%|██████████████████████████████████████████████████                             | 633/1000 [17:46<11:06,  1.82s/it]